### Set Data Path

In [1]:
from pathlib import Path

base_dir = Path("data")
train_dir = base_dir/Path("train")
validation_dir = base_dir/Path("validation")
test_dir = base_dir/Path("test")

### Image Transform Function

In [2]:
from torchvision import transforms

transform = transforms.Compose([ 
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(.5, .5, .5), std=(.5, .5, .5))
])

### Load Training Data (x: features, y: labels)

In [3]:
import torch
from PIL import Image

x, y = [], []
for file_name in train_dir.glob("*.jpg"):
    bounding_box_file = file_name.with_suffix('.txt')
    
    with open(bounding_box_file) as file:
        lines = file.readlines()
        if(len(lines) > 1):
            continue
        else:
            line = lines[0].strip('\n')
            (classes, cen_x, cen_y, box_w, box_h) = list(map(float, line.split(' ')))
            torch_data = torch.FloatTensor([cen_x, cen_y, box_w, box_h])
            y.append(torch_data)
            
    img = Image.open(str(file_name)).convert('RGB')
    img = transform(img)
    x.append(img)

### Put Training Data into Torch Loader

In [4]:
import torch.utils.data as Data

tensor_x = torch.stack(x)
tensor_y = torch.stack(y)
torch_dataset = Data.TensorDataset(tensor_x, tensor_y)
loader = Data.DataLoader(dataset=torch_dataset, batch_size=32, shuffle=True, num_workers=2)

### Load Pretrained RestNet18 Model

In [6]:
import torchvision
from torch import nn

model = torchvision.models.resnet18(pretrained=True)
fc_in_size = model.fc.in_features
model.fc = nn.Linear(fc_in_size, 4)

### Parameters

In [7]:
EPOCH = 10
LR = 1e-3

### Loss Function & Optimizer

In [8]:
loss_func = nn.SmoothL1Loss()
opt = torch.optim.Adam(model.parameters(), lr=LR)

### Training

In [ ]:
for epoch in range(EPOCH):
    for step, (batch_x, batch_y) in enumerate(loader):
        batch_x = batch_x
        batch_y = batch_y
        output = model(batch_x)
        loss = loss_func(output, batch_y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        if(step % 5 == 0):
            print("Epoch {} | Step {} | Loss {}".format(epoch, step, loss))

Epoch 0 | Step 0 | Loss 0.3441098928451538
Epoch 0 | Step 5 | Loss 0.17193390429019928
Epoch 0 | Step 10 | Loss 0.10625818371772766
Epoch 1 | Step 0 | Loss 0.0646333172917366
Epoch 1 | Step 5 | Loss 0.03650979697704315
Epoch 1 | Step 10 | Loss 0.017900140956044197


### Show some of the Prediction

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

model = model.cpu()
for batch_x, batch_y in loader:
    predict = model(batch_x)
    for x, pred, y in zip(batch_x, predict, batch_y):
        (pos_x, pos_y, box_w, box_h) = pred
        pos_x *= 224
        pos_y *= 224
        box_w *= 224
        box_h *= 224
        
        image = transforms.ToPILImage()(x)
        img = cv2.cvtColor(np.asarray(image), cv2.COLOR_RGB2BGR)
        img = cv2.rectangle(img, (pos_x - box_w/2, pos_y - box_h/2), (pos_x + box_w/2, pos_y + box_h/2), (255, 0, 0), 3)
        
        plt.imshow(img)
        plt.show()
    break